
# Sesión 08 — Clasificación con **Wine Quality (UCI)**

Este cuaderno implementa un flujo clásico de **clasificación** en Python usando el dataset **Wine Quality** de UCI (red + white).  
Incluye: carga de datos, EDA, preparación, división *train/test*, estandarización, ajuste de modelos (LogReg, Árbol, Random Forest, SVM), validación cruzada, métricas, matriz de confusión, **ROC** y **importancia de variables**.

> Requisitos: `pandas`, `numpy`, `scikit-learn`, `matplotlib`.



## 0) Descarga de datos (instrucciones)

Este notebook espera dos archivos en el mismo directorio o ruta accesible:

- `winequality-red.csv`
- `winequality-white.csv`

**Fuentes UCI** (copiar/pegar en el navegador):
- Red: https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv
- White: https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv

Colócalos en `./` o ajusta las rutas en la celda de carga. Si ya los tienes en esta sesión, ponlos en `/mnt/data/`.


## 1) Importaciones

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support, classification_report,
    confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

pd.set_option('display.precision', 3)
np.set_printoptions(precision=3, suppress=True)


## 2) Carga y unión de datos (red + white)

In [ ]:

# Rutas locales (ajustar si es necesario)
path_red = Path('data/winequality-red.csv')
path_white = Path('data/winequality-white.csv')

if not path_red.exists() or not path_white.exists():
    raise FileNotFoundError(
        "No se encontraron los CSVs de UCI. "
        "Descarga 'winequality-red.csv' y 'winequality-white.csv' de UCI y colócalos en la raiz del proyecto."
    )

# Los archivos de UCI usan ';' como separador
red = pd.read_csv(path_red, sep=';')
white = pd.read_csv(path_white, sep=';')

red['type'] = 'red'
white['type'] = 'white'

df = pd.concat([red], axis=0, ignore_index=True)
print(df.shape)
df.head()


## 3) Limpieza y creación de la variable objetivo

In [ ]:

# Comprobación de nulos
print("Nulos por columna:")
print(df.isna().sum())

# En este dataset no suele haber nulos; si hubiera, podemos imputar simple:
# df = df.fillna(df.median(numeric_only=True))

# Variable objetivo: umbral de calidad (clasificación binaria)
# Opción didáctica equilibrada: >= 6 como 'buena' (1), < 6 'no-buena' (0)
df['quality_label'] = (df['quality'] >= 6).astype(int)

# Separar X/y
features = [c for c in df.columns if c not in ['quality', 'quality_label']]
X = df[features].copy()
y = df['quality_label'].copy()

# One-hot para 'type' (red/white)
X = pd.get_dummies(X, columns=['type'], drop_first=True)

print("Dimensiones X:", X.shape, "| y:", y.shape)
print("Distribución de y:")
print(y.value_counts(normalize=True).rename('proportion'))


## 4) EDA rápida

In [ ]:
X.describe()


In [ ]:

# Histograma de la calidad original
plt.figure(figsize=(6,4))
plt.hist(df['quality'], bins=10)
plt.title("Distribución de 'quality' (0-10)")
plt.xlabel("quality")
plt.ylabel("Frecuencia")
plt.tight_layout()
plt.show()


## 5) Partición *train/test* (estratificada)

In [ ]:

# %%
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)
X_train.shape, X_test.shape


## 6) Entrenamiento y evaluación de modelos

In [ ]:

# Función para evaluar modelos
def evaluar_modelo(nombre, pipe, X_train, X_test, y_train, y_test, target_names=('No-buena','Buena')):
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_test, y_pred, average='binary', zero_division=0
    )
    cm = confusion_matrix(y_test, y_pred)
    print(f"\n=== {nombre} ===")
    print(f"Accuracy: {acc:.3f} | Precision: {prec:.3f} | Recall: {rec:.3f} | F1: {f1:.3f}")
    print("\nReporte de Clasificación:")
    print(classification_report(y_test, y_pred, target_names=target_names, zero_division=0))
    ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=target_names).plot()
    plt.title(f"Matriz de confusión — {nombre}")
    plt.tight_layout()
    plt.show()
    return {"name": nombre, "pipeline": pipe, "y_pred": y_pred, "acc": acc}


In [ ]:

# Modelo 1: Regresión Logística
logreg = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=1000))
])
res_logreg = evaluar_modelo("Logistic Regression", logreg, X_train, X_test, y_train, y_test)


In [ ]:
# Modelo 2: KNN
from sklearn.neighbors import KNeighborsClassifier
knn = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", KNeighborsClassifier(n_neighbors=5))
])
res_knn = evaluar_modelo("K-Nearest Neighbors", knn, X_train, X_test, y_train, y_test)

In [ ]:
# Modelo 2: Decision Tree
tree = DecisionTreeClassifier(random_state=42, max_depth=None)
res_tree = evaluar_modelo("Decision Tree", tree, X_train, X_test, y_train, y_test)


In [ ]:
# Modelo 3: Random Forest
rf = RandomForestClassifier(random_state=42, n_estimators=400, max_depth=None)
res_rf = evaluar_modelo("Random Forest", rf, X_train, X_test, y_train, y_test)


In [ ]:
# Modelo 4: SVM (RBF)
svm_rbf = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", SVC(kernel="rbf", probability=True, random_state=42))
])
res_svm = evaluar_modelo("SVM (RBF)", svm_rbf, X_train, X_test, y_train, y_test)


### 6.1) Validación cruzada (k=5, accuracy)

In [ ]:

# Validación cruzada estratificada (5 folds)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for nombre, pipe in [
    ("LogReg", logreg),
    ("DecisionTree", tree),
    ("RandomForest", rf),
    ("SVM-RBF", svm_rbf),
]:
    scores = cross_val_score(pipe, X, y, cv=cv, scoring="accuracy")
    print(f"{nombre}: mean={scores.mean():.3f} ± {scores.std():.3f}")


## 7) Curva ROC (binaria)

In [ ]:

# Curvas ROC para modelos con probabilidad
def plot_roc_bin(nombre, fitted_pipe, X_test, y_test):
    # Si el estimador soporta predict_proba usarlo; si no, decision_function
    if hasattr(fitted_pipe, "predict_proba"):
        y_score = fitted_pipe.predict_proba(X_test)[:, 1]
    else:
        # decision_function puede ser usado como score
        if hasattr(fitted_pipe, "decision_function"):
            y_score = fitted_pipe.decision_function(X_test)
            # Normalización min-max por si la escala es arbitraria
            y_score = (y_score - y_score.min()) / (y_score.max() - y_score.min() + 1e-12)
        else:
            raise AttributeError("El estimador no provee predict_proba ni decision_function.")
    fpr, tpr, _ = roc_curve(y_test, y_score)
    roc_auc = auc(fpr, tpr)
    plt.figure(figsize=(6,4))
    plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.3f}")
    plt.plot([0,1],[0,1], linestyle='--')
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.title(f"ROC — {nombre}")
    plt.legend()
    plt.tight_layout()
    plt.show()

# Ajustar y graficar para LogReg y SVM
logreg.fit(X_train, y_train)
plot_roc_bin("Logistic Regression", logreg, X_test, y_test)

svm_rbf.fit(X_train, y_train)
plot_roc_bin("SVM (RBF)", svm_rbf, X_test, y_test)


## 8) Importancia de variables (árboles/bosques)

In [ ]:

# Importancia de variables con Random Forest
rf_fit = RandomForestClassifier(random_state=42, n_estimators=400)
rf_fit.fit(X_train, y_train)
importances = rf_fit.feature_importances_
idx = np.argsort(importances)[::-1]

print("Top 12 variables por importancia:")
for i in idx[:12]:
    print(f"{X.columns[i]:30s} {importances[i]:.4f}")

plt.figure(figsize=(8,5))
top_k = len(X.columns)
plt.bar(range(top_k), importances[idx][:top_k])
plt.xticks(range(top_k), [X.columns[i] for i in idx[:top_k]], rotation=45, ha='right')
plt.ylabel("Importancia (Gini)")
plt.title("Importancia de variables — Random Forest")
plt.tight_layout()
plt.show()



## 9) Notas didácticas
- Umbral de calidad: puedes cambiarlo a `>= 7` para un problema más **desbalanceado** (alta precisión, menor recall).  
- Escalado: necesario para modelos sensibles a la escala (LogReg, SVM).  
- Considera comparar *accuracy* con *AUC-ROC* y *F1* cuando las clases estén desbalanceadas.  
- Para tareas multi-clase, puedes predecir la calidad exacta (0–10), pero muchas celdas deben adaptarse.
